In [1]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch
device = "cuda" if torch.cuda.is_available() else "cpu"

tokenizer = AutoTokenizer.from_pretrained("microsoft/prophetnet-large-uncased-cnndm")
model = AutoModelForSeq2SeqLM.from_pretrained("microsoft/prophetnet-large-uncased-cnndm").to(device)

In [2]:
device

'cuda'

In [3]:
import pandas as pd

In [4]:
df = pd.read_csv('/teamspace/studios/this_studio/NLP_project/openai_reddit_final.csv', sep=';')

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 83797 entries, 0 to 83796
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Unnamed: 0    83797 non-null  int64 
 1   user_id       83797 non-null  object
 2   doc_id        83797 non-null  object
 3   user_profile  83797 non-null  object
 4   post/article  83797 non-null  object
 5   summary_text  83797 non-null  object
 6   confidence    83797 non-null  int64 
dtypes: int64(2), object(5)
memory usage: 4.5+ MB


In [6]:
tokenizer, model

(ProphetNetTokenizer(name_or_path='microsoft/prophetnet-large-uncased-cnndm', vocab_size=30522, model_max_length=1000000000000000019884624838656, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[SEP]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
 	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
 	100: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
 	102: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
 	103: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
 },
 ProphetNetForConditionalGeneration(
   (prophetnet): ProphetNetModel(
     (word_embeddings): Embedding(30522, 1024, padding_idx=0)
     (encoder): ProphetNetEncoder(
    

In [7]:
data = df[['doc_id', 'post/article']]

In [8]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 83797 entries, 0 to 83796
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   doc_id        83797 non-null  object
 1   post/article  83797 non-null  object
dtypes: object(2)
memory usage: 1.3+ MB


In [9]:
mapp = {}

for idx , row in data.iterrows():
    doc_id = row['doc_id']
    text = row['post/article']

    if doc_id not in mapp:
        mapp[doc_id] = text

In [10]:
len(mapp)

6218

In [11]:
ids = list(mapp.keys())
len(ids)

6218

In [12]:
unique_texts = list(mapp.values())
len(unique_texts)

6218

In [38]:
batch_size = 7

def translate_batch(texts):  
    inputs = tokenizer(texts, max_length=100, return_tensors='pt', padding=True, truncation=True).to(device)
    summary_ids = model.generate(inputs['input_ids'], num_beams=4, max_length=512, early_stopping=True)

    summaries = tokenizer.batch_decode(summary_ids, skip_special_tokens=True)
    return summaries


In [1]:
results = []

In [ ]:
src_texts = unique_texts
len(src_texts)

In [43]:
for i in range(0, len(src_texts), batch_size):
    batch_texts = src_texts[i:i + batch_size]    
    translated_texts = translate_batch(batch_texts)
    results.extend(translated_texts)


In [44]:
len(results)

6218

In [45]:
gen_summaries = results

In [46]:
mapp = dict(zip(ids, gen_summaries))

In [47]:
len(mapp)

6218

In [53]:
df['prophetnet_model_summary'] = df['doc_id'].map(mapp)

In [54]:
df.to_csv('openai_reddit_final_prophetnet.csv', sep=';')

In [55]:
data = pd.read_csv('/teamspace/studios/this_studio/openai_reddit_final_prophetnet.csv', sep=';')